# Regularization

As we saw with the CNN competition, CNNs have the tendency to **overfit** to their dataset, where our CNNs had:
- high performance on the training dataset
- low performance on our validation dataset

But what does this mean? And how do we fight it?


## Variance and Overfitting

**Variance** is a characteristic of a machine learning model that describes how _variant_ a model based on the dataset it's trained on. A high variance model is one that is _super_ sensitive to input training data provided - literally removing a _single_ data point could change the trends learned by the model!

Models that are super complex (such as CNNs) often have high variance because their complexity allows them to learn super complicated patterns in the training data. This makes them _very_ sensitive to the training data.

A **high variance** model is not a problem on its own! However when a high variance model is combined with either:
- a dataset that is too small
- a noisy dataset which contains lots of outliers / trends that don't generalize well to all data points

we observe the problem called **overfitting** - a situation where a model hyperfixates on noisy trends in the training data and fails to generalize to new data in the wild. As a result, an overfitting model has strong training set performance but _very_ poor testing set performance.

To see an example of overfitting, consider the case study from [this popular research paper](https://arxiv.org/pdf/1807.04975.pdf). In the paper, the researchers trained a computer vision model to recognize entities (such as a cow 🐮).

The authors then tried to extend their model to novel situations - such as identifying a cow on the beach (🐮 🏝️). For each of 3 situations, the authors, compared the top 5 most likely classes as per the model's predictions, and reported the results below (image reproduced from Figure 1 in paper).

![Image of machine learning model training results for a cow in a pasture vs a cow on a beach.](https://miro.medium.com/v2/resize:fit:1400/0*EWVAp3dh3d5YwsGn.png)

The figure above shows a clear trend:
- for cows that appear in pastures, which is a common scenario, the model is very confident that the image contains a cow (99% confident)
- for cows that appear on a beach, the model has no clue that the image contains a cow

Why is this happening? **Overfitting**! The model in paper was trained _only_ on data that contained cows on pastures (as most cows usually do live on pastures). However as a result, the model learned to associate _pastures_ with _cows_ - which is a noisy trend! While cows may certainly tend to live on pastures, a cow need not live on a pasture to be a cow.

In other words, the model fit _too closely_ to noisy patterns in the training dataset (the presence of pastures for cows) and learned relationships that didn't generalize well to new data (like cows on the beach).

**KEY 🔑**: It's easy to write this off as a strange edge case, but overfitting is _extremely common_ for convolutional neural networks. These are really powerful models that very easily overfit to training data unless you have particularly clean data.

## Reducing Overfitting

Let's work through an example that will help us see how we can use deep learning methods to reduce the amount of overfitting. Let's start by training a super simple CNN on the [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset containing images of items of clothing.

Let's start by training the CNN on the dataset and observing the results! This will take about 4 minutes to complete.

------

**Hint**: Enable GPUs to speed this up! Select `Runtime > Change runtime type` above to switch to "T4 GPU". **CRITICAL** - make sure to switch back to CPU when done with this notebook (otherwise you might get banned from Google Colab 😬)

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

converter = ToTensor()

train = datasets.FashionMNIST(root="./private", download=True, train=True, transform=converter)
test = datasets.FashionMNIST(root="./private", download=True, train=False, transform=converter)

In [ ]:
from torch import nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(144*4, 10)

    def forward(self, img):
        x1 = self.pool(self.relu(self.conv1(img)))
        x2 = self.pool(self.relu(self.conv2(x1)))
        x3 = self.pool(self.relu(self.conv3(x2)))
        flat = self.flatten(x3)
        return self.dense(flat)

In [ ]:
from typing import Callable
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl

device = "cuda" if torch.cuda.is_available() else "cpu"
seaborn.set_theme()

def evaluate(
    model: torch.nn.Module, criterion: Callable, loader: DataLoader
) -> tuple[float]:
    model.eval()
    with torch.no_grad():
        correct, total = 0, 0
        loss = 0.0
        for X, y in loader:
            outputs = model(X.to(device)).to("cpu")
            loss += criterion(outputs, y).detach().sum().item()
            _, predicted = torch.max(outputs.data, 1)  # get predicted digit
            total += len(y)
            correct += (predicted == y).sum().item()
    model.train()
    return correct / total, loss / total

def train_model(
    model: torch.nn.Module,
    criterion: Callable,
    optimizer: torch.optim.Optimizer,
    train_loader: DataLoader,
    test_loader: DataLoader,
    epochs: int = 10,
) -> dict[str, list[float]]:
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    model = model.to(device)

    for epoch in range(epochs):
        model.train()

        for X, y in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(X.to(device))
            loss = criterion(outputs, y.to(device))
            loss.backward()
            optimizer.step()

        train_accuracy, train_loss = evaluate(model, criterion, train_loader)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        test_accuracy, test_loss = evaluate(model, criterion, test_loader)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        print(
            f"Epoch {epoch + 1}: Loss - (Train {train_loss:.2f}/Test {test_loss:.2f}, "
            f"Accuracy - (Train {train_accuracy:.2f}/Test {test_accuracy:.2f})"
        )

    return {
        "loss": {
            "train": train_losses,
            "test": test_losses,
        },
        "accuracy": {
            "train": train_accuracies,
            "test": test_accuracies,
        },
    }


def plot_performance(history: dict[str, dict[str, list[float]]]) -> mpl.figure.Figure:
    fig, axes = plt.subplots(len(history), 1, figsize=(10, 8))
    for i, (metric, values) in enumerate(history.items()):
        train, test = values["train"], values["test"]
        axes[i].plot(train, label="train")
        axes[i].plot(test, label="test")
        axes[i].set_title(f"{metric}")
        axes[i].set_xlabel("Epoch")
        axes[i].set_ylabel(metric)
        axes[i].legend()

In [ ]:
train_loader = DataLoader(train, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

history = train_model(model, criterion, optimizer, train_loader, test_loader, epochs=5)

Now let's plot our results!

In [ ]:
plot_performance(history)

Notice the gap between the training and testing performance - that's a clear sign of overfitting (as our model is doing better on the training set than on the testing set).

Given that our model is overfitting, let's go over ways to reduce the amount of model overfitting. There are three ways (among many others) with which we will try and do so:
1. Reduce model complexity
2. Add dropout
3. Add weight decay

### Reducing Model Complexity

We say that a model is **complex** when it can learn more complicated patterns. For a CNN, model complexity is mainly a function of the number of layers and number of filters.

As a less complex model is less prone to overfitting, lets define a new CNN architecture that reduces the number of filters and see how our performance changes:

| Layer | Old Filter Count | New Filter Count |
| ----- | ---------------- | ---------------- |
| 1     | 16               | 8                |
| 2     | 32               | 8                |
| 3     | 64               | 16               |

In [ ]:
from torch import nn

class LessComplexCNN(nn.Module):
    def __init__(self):
        super(LessComplexCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(144, 10)

    def forward(self, img):
        x1 = self.pool(self.relu(self.conv1(img)))
        x2 = self.pool(self.relu(self.conv2(x1)))
        x3 = self.pool(self.relu(self.conv3(x2)))
        flat = self.flatten(x3)
        return self.dense(flat)

In [ ]:
train_loader = DataLoader(train, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

model = LessComplexCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

history = train_model(model, criterion, optimizer, train_loader, test_loader, epochs=5)

In [ ]:
plot_performance(history)

Now notice how the gap is smaller! Our model now overfits less to the training data (of course, at the cost of worse train accuracy).

**KEY 🔑**: This is a downside of reducing model complexity - by reducing the model complexity, you reduce the ability for the model to learn trends in the dataset, reducing training set performance. This is called the **bias-variance tradeoff**, and is a key problem in machine learning.

### Adding Dropout

Another way to reduce model complexity (than just reducing the number of filters) is to add something called **dropout** as a layer in your model.

Dropout is a special layer that just takes the inputs, and **randomly** delets some percentage of the values by setting them to 0. Deleting inputs seems counterintuitive, but the fact that dropout is done _randomly_ forces the most to pay less attention to noise.

If you want to learn more, checkout the original [research paper](https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf).

The cell below adds dropout to our original CNN architecture.

In [ ]:
from torch import nn

class DropoutCNN(nn.Module):
    def __init__(self):
        super(DropoutCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(144*4, 10)

        # this is new!
        self.dropout = nn.Dropout(0.5)

    def forward(self, img):
        x1 = self.pool(self.relu(self.conv1(img)))
        x2 = self.pool(self.relu(self.conv2(x1)))
        x3 = self.pool(self.relu(self.conv3(x2)))
        flat = self.flatten(x3)
        flat_droppedout = self.dropout(flat) # we usually apply dropout to linear layers only
        return self.dense(flat_droppedout)

In [ ]:
train_loader = DataLoader(train, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

model = DropoutCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

history = train_model(model, criterion, optimizer, train_loader, test_loader, epochs=5)

In [ ]:
plot_performance(history)

Here too - we see the gap between training and testing loss decrease! Except now, we don't give up as much training accuracy to reduce overfitting!

### Adding Weight Decay

Yet another way to reduce model complexity (than just reducing the number of filters) is to use something called **weight decay**.

Unlike dropout, which is its own special layer in a neural network, weight decay is part of the _optimizer_. It essentially tells the optimizer to add a penalty if the model is too complex. Therefore when training the CNN, we make sure to train a model that has lower complexity.

The cell below edits the definition of our optimizer to add weight decay. We will use `AdamW` instead of `Adam` now to make sure weight decay works properly.

In [ ]:
train_loader = DataLoader(train, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-3, weight_decay=0.1) # this is new!

history = train_model(model, criterion, optimizer, train_loader, test_loader, epochs=5)

In [ ]:
plot_performance(history)

Yet again, we see that a smaller gap between the train and validation curves, showing that the amount of overfitting is decreasing.